In [1]:
#import depedencies
import pandas as pd
import fiona
import folium
import geopandas as gpd
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
import math

In [2]:
#Creating path to import csv files into Pandas
Obesity_df = pd.read_csv("../data/Merged_data/3. Obesity-prevalence-by-state-2021.csv")
df_Pop = pd.read_csv("../data/Merged_data/Pop_dataUS.csv")
states_df = pd.read_csv("../data/statelatlong.csv")
#Renaming column in states_df to merge with other dataframes
states_df.rename(columns = {'City':'State'}, inplace = True)

In [10]:
#dropping all NaN values for Co-ordinates
ob_df = Obesity_df.dropna()
ob_df.head()

,State,,Prevalence,95% CI,Latitude,Longitude,City
0,Alabama,AL,39.9,"(37.9, 41.8)",32.601011,-86.680736,Alabama
1,Alaska,AK,33.5,"(31.5, 35.5)",61.302501,-158.775020,Alaska
2,Arizona,AZ,31.3,"(30.0, 32.6)",34.168219,-111.930907,Arizona
3,Arkansas,AR,38.7,"(36.6, 40.8)",34.751928,-92.131378,Arkansas
4,California,CA,27.6,"(26.1, 29.1)",37.271875,-119.270415,California


In [4]:
df_Pop.head()

,SUMLEV,REGION,DIVISION,STATE,State_abbreviations,NAME,ESTIMATESBASE2020,POPESTIMATE2020,POPESTIMATE2021,NPOPCHG_2020,...,RBIRTH2021,RDEATH2021,RNATURALINC2021,RINTERNATIONALMIG2021,RDOMESTICMIG2021,RNETMIG2021,Unnamed: 31,Latitude,Longitude,City
0,40,3,6,1,AL,Alabama,5024279,5024803,5039877,524,...,11.191613,12.890226,-1.698613,0.247201,4.398749,4.645950,NaN,32.601011,-86.680736,Alabama
1,40,4,9,2,AK,Alaska,733391,732441,732673,-950,...,12.667956,7.700425,4.967532,0.718033,-5.295151,-4.577118,NaN,61.302501,-158.775020,Alaska
2,40,4,8,4,AZ,Arizona,7151502,7177986,7276316,26484,...,10.584669,10.469547,0.115121,0.619608,12.871739,13.491347,NaN,34.168219,-111.930907,Arizona
3,40,3,7,5,AR,Arkansas,3011524,3012232,3025891,708,...,11.599962,12.671819,-1.071856,0.272932,5.304960,5.577892,NaN,34.751928,-92.131378,Arkansas
4,40,4,9,6,CA,California,39538223,39499738,39237836,-38485,...,10.778412,8.441637,2.336775,0.364223,-9.329701,-8.965478,NaN,37.271875,-119.270415,California


In [5]:
#Selecting columns for Data analysis with just the 2020 Population Data
df_pop2 = df_Pop.iloc[:, [4,7,32,33,34]]
df_pop3 = df_pop2.dropna()
df_pop3.head()


,State_abbreviations,POPESTIMATE2020,Latitude,Longitude,City
0,AL,5024803,32.601011,-86.680736,Alabama
1,AK,732441,61.302501,-158.775020,Alaska
2,AZ,7177986,34.168219,-111.930907,Arizona
3,AR,3012232,34.751928,-92.131378,Arkansas
4,CA,39499738,37.271875,-119.270415,California


In [6]:
#Using the state co-ordinates to create a heat map of Population
states_df.head()

,State,Latitude,Longitude,State
0,AL,32.601011,-86.680736,Alabama
1,AK,61.302501,-158.775020,Alaska
2,AZ,34.168219,-111.930907,Arizona
3,AR,34.751928,-92.131378,Arkansas
4,CA,37.271875,-119.270415,California


In [7]:
#creating the base map
#creating layer with population estimates on map
# Add a HeatMap map to the base map
lats_longs_weight = list(map(list, zip(df_pop3["Latitude"],
                          df_pop3["Longitude"],
                          df_pop3["POPESTIMATE2020"]
                         )
               )
           )

m= folium.Map(location = [38.27312, -98.5821872], zoom_start = 4)
#Creating a title for the layers in map
fg = folium.FeatureGroup(name="Population Estimates 2020")
fg.add_child(HeatMap(lats_longs_weight))
#adding the featuregroup to map
m.add_child(fg)
m

In [8]:
for i,row in ob_df.iterrows():
    #Setup the content of the popup
    iframe = folium.IFrame(f'State Name: {str(row["State"])} with Obesity %: {str(row["Prevalence"])}')
    
    #Initialise the tootip using the iframe
    popup = folium.Popup(iframe, min_width=150, max_width=250)
       
    #Add each row to the map
    m.add_child(folium.Marker(location=[row['Latitude'],row['Longitude']],
                  popup = popup))
                           
m.add_child(folium.LayerControl())

In [9]:
#Saving map to html
# m.save("Population_vs_Obesity.html")